In [1]:
import numpy as np
import pandas as pd
import datetime
from datetime import date, datetime
import gc
import matplotlib.pyplot as plt
import seaborn as sns
import lightgbm as lgb
from sklearn.model_selection import StratifiedKFold
from sklearn.metrics import mean_squared_error
import warnings


warnings.filterwarnings('ignore')
np.random.seed(4590)

skf= StratifiedKFold(n_splits=5, shuffle=True, random_state=4590)

# by feature select 
FILTER_FEATURE = ['new_hist_month_diff_min_-_hist_month_diff_min', 'new_hist_merchant_category_id_mean_mean_add_hist_merchant_category_id_mean_mean', 'new_hist_month_lag_mean_mean_add_hist_month_lag_mean_mean', 'hist_year_nunique', 'new_hist_month_diff_max_/_hist_month_diff_max', 'new_hist_merchant_id_mean_mean_add_hist_merchant_id_mean_mean', 'new_hist_month_diff_min_/_hist_month_diff_min', 'new_hist_year_mean_mean_add_hist_year_mean_mean', 'new_hist_dayofweek_mean_mean_/_hist_dayofweek_mean_mean', 'new_hist_weekofyear_mean_mean_add_hist_weekofyear_mean_mean', 'new_hist_dayofweek_nunique_/_hist_dayofweek_nunique', 'new_hist_month_diff_max', 'new_hist_year_mean_mean_/_hist_year_mean_mean', 'new_hist_merchant_category_id_mean_mean_/_hist_merchant_category_id_mean_mean', 'new_hist_merchant_id_mean_mean_/_hist_merchant_id_mean_mean', 'new_hist_year_nunique_/_hist_year_nunique', 'new_hist_month_diff_min', 'new_hist_merchant_category_id_mean_mean', 'hist_month_diff_mean_hist_month_diff_min_ctr', 'new_hist_year_nunique_-_hist_year_nunique', 'dayofweek', 'new_hist_month_mean_mean_/_hist_month_mean_mean', 'new_hist_month_diff_var', 'new_hist_installments_min_-_hist_installments_min', 'new_hist_subsector_id_mean_mean', 'new_hist_dayofweek_nunique', 'new_hist_weekend_mean', 'new_hist_category_2_mean_mean_/_hist_category_2_mean_mean', 'new_hist_merchant_category_id_nunique_-_hist_merchant_category_id_nunique', 'new_hist_month_diff_mean_add_hist_month_diff_mean', 'new_hist_authorized_flag_mean', 'new_hist_merchant_category_id_mean_mean_-_hist_merchant_category_id_mean_mean', 'new_hist_year_mean_mean_-_hist_year_mean_mean', 'new_hist_category_2_mean_mean', 'new_hist_card_id_mean_mean_add_hist_card_id_mean_mean', 'new_hist_month_diff_max_-_hist_month_diff_max', 'new_hist_weekend_sum']

In [2]:
df_train = pd.read_csv('../input/train.csv')
df_test = pd.read_csv('../input/test.csv')
df_hist_trans = pd.read_csv('../input/historical_transactions.csv',parse_dates=['purchase_date'])
df_new_merchant_trans = pd.read_csv('../input/new_merchant_transactions.csv',parse_dates=['purchase_date'])

In [3]:
max_date=df_new_merchant_trans['purchase_date'].max()

In [4]:
def reduce_mem_usage(df, verbose=True):
    numerics = ['int16', 'int32', 'int64', 'float16', 'float32', 'float64']
    start_mem = df.memory_usage().sum() / 1024**2    
    for col in df.columns:
        col_type = df[col].dtypes
        if col_type in numerics:
            c_min = df[col].min()
            c_max = df[col].max()
            if str(col_type)[:3] == 'int':
                if c_min > np.iinfo(np.int8).min and c_max < np.iinfo(np.int8).max:
                    df[col] = df[col].astype(np.int8)
                elif c_min > np.iinfo(np.int16).min and c_max < np.iinfo(np.int16).max:
                    df[col] = df[col].astype(np.int16)
                elif c_min > np.iinfo(np.int32).min and c_max < np.iinfo(np.int32).max:
                    df[col] = df[col].astype(np.int32)
                elif c_min > np.iinfo(np.int64).min and c_max < np.iinfo(np.int64).max:
                    df[col] = df[col].astype(np.int64)  
            else:
                if c_min > np.finfo(np.float16).min and c_max < np.finfo(np.float16).max:
                    df[col] = df[col].astype(np.float16)
                elif c_min > np.finfo(np.float32).min and c_max < np.finfo(np.float32).max:
                    df[col] = df[col].astype(np.float32)
                else:
                    df[col] = df[col].astype(np.float64)    
    end_mem = df.memory_usage().sum() / 1024**2
    if verbose: print('Mem. usage decreased to {:5.2f} Mb ({:.1f}% reduction)'.format(end_mem, 100 * (start_mem - end_mem) / start_mem))
    return df

In [5]:
df_train = reduce_mem_usage(df_train)
df_test = reduce_mem_usage(df_test)
df_hist_trans = reduce_mem_usage(df_hist_trans)
df_new_merchant_trans = reduce_mem_usage(df_new_merchant_trans)
gc.collect()

Mem. usage decreased to  4.04 Mb (56.2% reduction)
Mem. usage decreased to  2.24 Mb (52.5% reduction)
Mem. usage decreased to 1749.11 Mb (43.7% reduction)
Mem. usage decreased to 114.20 Mb (45.5% reduction)


0

In [6]:
df_train.head()

,first_active_month,card_id,feature_1,feature_2,feature_3,target
0,2017-06,C_ID_92a2005557,5,2,1,-0.820312
1,2017-01,C_ID_3d0044924f,4,1,0,0.392822
2,2016-08,C_ID_d639edf6cd,2,2,0,0.687988
3,2017-09,C_ID_186d6a6901,4,3,0,0.142456
4,2017-11,C_ID_cdbd2c0db2,1,3,0,-0.159790


In [7]:
def get_new_columns(name,aggs):
    return [name + '_' + k + '_' + agg for k in aggs.keys() for agg in aggs[k]]

In [8]:

for df in [df_hist_trans,df_new_merchant_trans]:
#     df['purchase_date'] = pd.to_datetime(df['purchase_date'])
    df['year'] = df['purchase_date'].dt.year
    df['weekofyear'] = df['purchase_date'].dt.weekofyear
    df['month'] = df['purchase_date'].dt.month
    df['dayofweek'] = df['purchase_date'].dt.dayofweek
    df['weekend'] = (df.purchase_date.dt.weekday >=5).astype(int)
    df['hour'] = df['purchase_date'].dt.hour
    df['authorized_flag'] = df['authorized_flag'].map({'Y':1, 'N':0})
    df['category_1'] = df['category_1'].map({'Y':1, 'N':0}) 
    df['month_diff'] = ((max_date - df['purchase_date']).dt.days)//30
    df['month_diff'] += df['month_lag']
#     df['authorized_flag_purchase_amount'] = df.apply(lambda x: x['purchase_amount'] \
#                             if x['authorized_flag']>0 else 0 ,axis=1)
    
#     df['unauthorized_flag_purchase_amount'] = df.apply(lambda x: x['purchase_amount'] \
#                             if x['authorized_flag']<1 else 0 ,axis=1)
    
    
    
    
    
                                                                           

In [9]:
def  get_agg_fea(count_df,prefix):
    aggs = {}
    for col in ['month','hour','weekofyear','dayofweek','year','subsector_id','merchant_id','merchant_category_id']:
        aggs[col] = ['nunique']

    aggs['purchase_amount'] = ['sum','max','min','mean','var']
    aggs['installments'] = ['sum','max','min','mean','var']
    aggs['purchase_date'] = ['max','min']
    aggs['month_lag'] = ['max','min','mean','var']
    # aggs['month_diff'] = ['mean']
    aggs['month_diff'] = ['max','min','mean','var']
    aggs['authorized_flag'] = ['sum', 'mean']
    aggs['weekend'] = ['sum', 'mean']
    aggs['category_1'] = ['sum', 'mean']
    aggs['card_id'] = ['size']

    for col in ['month','hour','weekofyear','dayofweek','year','subsector_id','merchant_id','merchant_category_id',\
                'category_1','category_2','category_3','month_lag','card_id']:
        count_df[col+'_mean'] = count_df.groupby([col])['purchase_amount'].transform('mean')
        aggs[col+'_mean'] = ['mean']   

    new_columns = get_new_columns(prefix,aggs)
    count_df_gp = count_df.groupby('card_id').agg(aggs)
    count_df_gp.columns = new_columns
    count_df_gp.reset_index(drop=False,inplace=True)
    count_df_gp['%s_purchase_date_diff'%prefix] = (count_df_gp['%s_purchase_date_max'%prefix] - count_df_gp['%s_purchase_date_min'%prefix]).dt.days
    count_df_gp['%s_purchase_date_average'%prefix] = count_df_gp['%s_purchase_date_diff'%prefix]/count_df_gp['%s_card_id_size'%prefix]
    count_df_gp['%s_purchase_date_uptonow'%prefix] = (max_date - count_df_gp['%s_purchase_date_max'%prefix]).dt.days
    return count_df_gp



hist_count_df=get_agg_fea(df_hist_trans,'hist')
df_train = df_train.merge(hist_count_df,on='card_id',how='left')
df_test = df_test.merge(hist_count_df,on='card_id',how='left')
del hist_count_df
gc.collect()
new_count_df = get_agg_fea(df_new_merchant_trans,'new_hist')
df_train = df_train.merge(new_count_df,on='card_id',how='left')
df_test = df_test.merge(new_count_df,on='card_id',how='left')
del new_count_df
gc.collect()

140

In [10]:
new_hist_feature=[ i for i in df_train  if 'new_hist' in i]

for fea in new_hist_feature:
    if fea in ['new_hist_purchase_date_max','new_hist_purchase_date_min']:
        continue
    print(fea)
    new_fea=fea
    hist_fea=fea.replace("new_hist",'hist')
    df_train[new_fea+"_add_"+hist_fea]=df_train[new_fea]+df_train[hist_fea]
    df_test[new_fea+"_add_"+hist_fea]=df_test[new_fea]+df_test[hist_fea]
    df_train[new_fea+"_-_"+hist_fea]=df_train[new_fea]-df_train[hist_fea]
    df_test[new_fea+"_-_"+hist_fea]=df_test[new_fea]-df_test[hist_fea]
    df_train[new_fea+"_/_"+hist_fea]=df_train[new_fea]/(df_train[hist_fea]+0.1)
    df_test[new_fea+"_/_"+hist_fea]=df_test[new_fea]/(df_test[hist_fea]+0.1)

new_hist_month_nunique
new_hist_hour_nunique
new_hist_weekofyear_nunique
new_hist_dayofweek_nunique
new_hist_year_nunique
new_hist_subsector_id_nunique
new_hist_merchant_id_nunique
new_hist_merchant_category_id_nunique
new_hist_purchase_amount_sum
new_hist_purchase_amount_max
new_hist_purchase_amount_min
new_hist_purchase_amount_mean
new_hist_purchase_amount_var
new_hist_installments_sum
new_hist_installments_max
new_hist_installments_min
new_hist_installments_mean
new_hist_installments_var
new_hist_month_lag_max
new_hist_month_lag_min
new_hist_month_lag_mean
new_hist_month_lag_var
new_hist_month_diff_max
new_hist_month_diff_min
new_hist_month_diff_mean
new_hist_month_diff_var
new_hist_authorized_flag_sum
new_hist_authorized_flag_mean
new_hist_weekend_sum
new_hist_weekend_mean
new_hist_category_1_sum
new_hist_category_1_mean
new_hist_card_id_size
new_hist_month_mean_mean
new_hist_hour_mean_mean
new_hist_weekofyear_mean_mean
new_hist_dayofweek_mean_mean
new_hist_year_mean_mean
new_hist_

In [11]:
# del df_hist_trans;
# gc.collect()

# del df_new_merchant_trans;
# gc.collect()

df_train.head(5)

,first_active_month,card_id,feature_1,feature_2,feature_3,target,hist_month_nunique,hist_hour_nunique,hist_weekofyear_nunique,hist_dayofweek_nunique,...,new_hist_card_id_mean_mean_/_hist_card_id_mean_mean,new_hist_purchase_date_diff_add_hist_purchase_date_diff,new_hist_purchase_date_diff_-_hist_purchase_date_diff,new_hist_purchase_date_diff_/_hist_purchase_date_diff,new_hist_purchase_date_average_add_hist_purchase_date_average,new_hist_purchase_date_average_-_hist_purchase_date_average,new_hist_purchase_date_average_/_hist_purchase_date_average,new_hist_purchase_date_uptonow_add_hist_purchase_date_uptonow,new_hist_purchase_date_uptonow_-_hist_purchase_date_uptonow,new_hist_purchase_date_uptonow_/_hist_purchase_date_uptonow
0,2017-06,C_ID_92a2005557,5,2,1,-0.820312,9,23,35,7,...,1.069365,296.0,-188.0,0.223048,3.278595,1.417057,2.277742,65.0,-63.0,0.015601
1,2017-01,C_ID_3d0044924f,4,1,0,0.392822,12,24,50,7,...,1.452096,446.0,-334.0,0.143553,10.447619,8.219048,7.686275,120.0,-58.0,0.347924
2,2016-08,C_ID_d639edf6cd,2,2,0,0.687988,10,14,22,7,...,1.210758,412.0,-412.0,0.000000,9.581395,-9.581395,0.000000,64.0,-60.0,0.032206
3,2017-09,C_ID_186d6a6901,4,3,0,0.142456,6,16,20,7,...,1.225325,195.0,-113.0,0.266061,7.857143,3.857143,2.789116,73.0,-49.0,0.196399
4,2017-11,C_ID_cdbd2c0db2,1,3,0,-0.159790,4,22,17,7,...,2.081052,165.0,-51.0,0.527290,2.395363,0.771303,1.736054,63.0,-59.0,0.032733


In [12]:
df_train['outliers'] = 0
df_train.loc[df_train['target'] < -30, 'outliers'] = 1
df_train['outliers'].value_counts()

0    199710
1      2207
Name: outliers, dtype: int64

In [13]:
def get_data_ctr_fea(tj_data,self_data,items=['feature_1','feature_2','feature_3','new_hist_month_diff_mean',
                                          'hist_month_diff_mean', 'hist_month_diff_max','hist_month_diff_min' ]):

    items = items
    
    tj_drop_columns=[i for i in  tj_data.columns  if "_ctr" in i]
    if len(tj_drop_columns)>0:
        tj_data=tj_data.drop(columns=tj_drop_columns)
    
    self_drop_columns=[i for i in  self_data  if "_ctr" in i]
    if len(self_drop_columns)>0:
        print(self_drop_columns)
        self_data.drop(columns=self_drop_columns,inplace=True)
        
    
    for item in items:
        if type(item)==list:
            pr_name="_".join(item)
            merge_columns=item+[pr_name+'_ctr']
        else:
            pr_name=item
            merge_columns=[item,pr_name+'_ctr']
        temp = tj_data.groupby(item, as_index = False)['outliers'].agg({pr_name+'_click':'sum',pr_name+'_count':'count'})
        temp[pr_name+'_ctr'] =1000* (temp[pr_name+'_click']+0.01)/(temp[pr_name+'_count']+0.01)
        
        self_data = pd.merge(self_data, temp[merge_columns], on=item, how='left')
        
    for i in range(len(items)):
        for j in range(i+1, len(items)):
            item_g = [items[i], items[j]]
            merge_columns=item_g+['_'.join(item_g)+'_ctr']
            temp = tj_data.groupby(item_g, as_index=False)['outliers'].agg({'_'.join(item_g)+'_click': 'sum','_'.join(item_g)+'count':'count'})
            temp['_'.join(item_g)+'_ctr'] =1000* (temp['_'.join(item_g)+'_click']+0.01)/(temp['_'.join(item_g)+'count']+0.01)
            self_data = pd.merge(self_data, temp[merge_columns], on=item_g, how='left')
            
    return self_data
            
df_test=get_data_ctr_fea(df_train,df_test)  
print("df_test ctr caluate finished")

from sklearn.model_selection import  StratifiedKFold
skf = StratifiedKFold(n_splits=5, random_state=4590, shuffle=True)

df_train['index'] = [i for i in range(len(df_train))]

for k, (df_train_in, df_test_in) in enumerate(skf.split(df_train, df_train.outliers.values)):
    df_train_df=df_train.iloc[df_train_in]
    val_df=df_train.iloc[df_test_in]
    val_df=get_data_ctr_fea(df_train_df,val_df)
    if k==0:
        new_df_train_df=val_df
    else:
        new_df_train_df=pd.concat([new_df_train_df,val_df])

df_train=new_df_train_df
df_train.sort_values(by='index',inplace=True)
df_train.drop(columns = ['index'],inplace=True)

print("df_train ctr caluate finished")


df_test ctr caluate finished
df_train ctr caluate finished


In [14]:
df_train.head()

,first_active_month,card_id,feature_1,feature_2,feature_3,target,hist_month_nunique,hist_hour_nunique,hist_weekofyear_nunique,hist_dayofweek_nunique,...,feature_3_new_hist_month_diff_mean_ctr,feature_3_hist_month_diff_mean_ctr,feature_3_hist_month_diff_max_ctr,feature_3_hist_month_diff_min_ctr,new_hist_month_diff_mean_hist_month_diff_mean_ctr,new_hist_month_diff_mean_hist_month_diff_max_ctr,new_hist_month_diff_mean_hist_month_diff_min_ctr,hist_month_diff_mean_hist_month_diff_max_ctr,hist_month_diff_mean_hist_month_diff_min_ctr,hist_month_diff_max_hist_month_diff_min_ctr
0,2017-06,C_ID_92a2005557,5,2,1,-0.820312,9,23,35,7,...,2.703738,9.900990,3.724646,3.625798,9.900990,2.778752,2.731318,3.322259,9.900990,3.909765
1,2017-01,C_ID_3d0044924f,4,1,0,0.392822,12,24,50,7,...,10.125030,NaN,18.765042,3.082772,NaN,11.680257,11.453054,NaN,NaN,14.906786
0,2016-08,C_ID_d639edf6cd,2,2,0,0.687988,10,14,22,7,...,1.868991,0.185151,2.876681,3.068904,0.086949,2.707610,2.773063,0.072987,0.083326,3.923032
2,2017-09,C_ID_186d6a6901,4,3,0,0.142456,6,16,20,7,...,1.792096,0.357015,2.856462,0.868094,16.287696,2.778752,0.749600,14.025830,0.666223,1.170184
1,2017-11,C_ID_cdbd2c0db2,1,3,0,-0.159790,4,22,17,7,...,1.868991,0.999001,2.876681,0.872406,0.768640,2.707610,0.750575,0.624610,1.426534,1.013272


In [15]:
# Dealing with the one nan in df_test.first_active_month a bit arbitrarily for now
df_test.loc[df_test['first_active_month'].isna(),'first_active_month'] = df_test.iloc[11577]['first_active_month']

In [16]:
for df in [df_train,df_test]:
    
    
    df['first_active_month'] = pd.to_datetime(df['first_active_month'])
    df['dayofweek'] = df['first_active_month'].dt.dayofweek
    df['weekofyear'] = df['first_active_month'].dt.weekofyear
    df['month'] = df['first_active_month'].dt.month
    df['elapsed_time'] = (max_date- df['first_active_month']).dt.days
    df['hist_first_buy'] = (df['hist_purchase_date_min'] - df['first_active_month']).dt.days
    df['new_hist_first_buy'] = (df['new_hist_purchase_date_min'] - df['first_active_month']).dt.days
    for f in ['hist_purchase_date_max','hist_purchase_date_min','new_hist_purchase_date_max',\
                     'new_hist_purchase_date_min']:
        df[f] = df[f].astype(np.int64) * 1e-9
    df['card_id_total'] = df['new_hist_card_id_size']+df['hist_card_id_size']
    df['purchase_amount_total'] = df['new_hist_purchase_amount_sum']+df['hist_purchase_amount_sum']

    
    df['purchase_amount_diff'] = df['new_hist_purchase_amount_sum']-df['hist_purchase_amount_sum']
    df['purchase_amount_rate'] = df['purchase_amount_diff']/df['hist_purchase_amount_sum']

  

In [17]:
df_train = reduce_mem_usage(df_train)
df_test = reduce_mem_usage(df_test)

Mem. usage decreased to 128.82 Mb (66.0% reduction)
Mem. usage decreased to 77.58 Mb (66.3% reduction)


In [18]:
# bed_fea =['new_hist_month_diff_min', 'new_hist_authorized_flag_mean', 'new_hist_year_mean_mean', 'new_hist_year_nunique_/_hist_year_nunique', 'new_hist_year_nunique_-_hist_year_nunique']
del_col =  ['card_id', 'first_active_month','target','outliers']+FILTER_FEATURE

features = [c for c in df_train.columns if c not in del_col ]

target = df_train['target']

train_y = target 
# df_train.drop(columns  = ["card_id","target"],inplace=True)
# df_test.drop(columns = ["card_id"],inplace=True)

In [19]:
print(len(features))

253


In [20]:
features

['feature_1',
 'feature_2',
 'feature_3',
 'hist_month_nunique',
 'hist_hour_nunique',
 'hist_weekofyear_nunique',
 'hist_dayofweek_nunique',
 'hist_subsector_id_nunique',
 'hist_merchant_id_nunique',
 'hist_merchant_category_id_nunique',
 'hist_purchase_amount_sum',
 'hist_purchase_amount_max',
 'hist_purchase_amount_min',
 'hist_purchase_amount_mean',
 'hist_purchase_amount_var',
 'hist_installments_sum',
 'hist_installments_max',
 'hist_installments_min',
 'hist_installments_mean',
 'hist_installments_var',
 'hist_purchase_date_max',
 'hist_purchase_date_min',
 'hist_month_lag_max',
 'hist_month_lag_min',
 'hist_month_lag_mean',
 'hist_month_lag_var',
 'hist_month_diff_max',
 'hist_month_diff_min',
 'hist_month_diff_mean',
 'hist_month_diff_var',
 'hist_authorized_flag_sum',
 'hist_authorized_flag_mean',
 'hist_weekend_sum',
 'hist_weekend_mean',
 'hist_category_1_sum',
 'hist_category_1_mean',
 'hist_card_id_size',
 'hist_month_mean_mean',
 'hist_hour_mean_mean',
 'hist_weekofyear_

In [21]:
param = {'num_leaves': 31,
         'min_data_in_leaf': 30, 
         'objective':'regression',
         'max_depth': -1,
         'learning_rate': 0.01,
         "min_child_samples": 30,
         "boosting": "gbdt",
         "feature_fraction": 0.5,
         "bagging_freq": 1,
         "bagging_fraction": 0.9 ,
         "metric": 'rmse',
         "lambda_l1": 0.1,
         'lambda_l2':0.1,
         "verbosity": -1,
         "n_jobs":10,
         "random_state": 4590}
NFOLD = 5
oof_train = np.zeros((len(df_train),1))
oof_test = np.zeros((len(df_test),1))
oof_test_skf = np.zeros((NFOLD,len(df_test),))

feature_importance_df = pd.DataFrame()

for fold_, (trn_idx, val_idx) in enumerate(skf.split(df_train,df_train['outliers'].values)):
    print("fold {}".format(fold_))
    trn_data = lgb.Dataset(df_train.iloc[trn_idx][features],target.iloc[trn_idx])#ategorical_feature=categorical_feats)
    val_data = lgb.Dataset(df_train.iloc[val_idx][features],target.iloc[val_idx])

    num_round = 10000
    clf = lgb.train(param, trn_data, num_round, valid_sets = [trn_data, val_data], verbose_eval=200, early_stopping_rounds = 500)
    oof_train[val_idx] = clf.predict(df_train.iloc[val_idx][features], num_iteration=clf.best_iteration).reshape(-1,1)
    
    fold_importance_df = pd.DataFrame()
    fold_importance_df["Feature"] = features
    fold_importance_df["importance"] = clf.feature_importance()
    fold_importance_df["fold"] = fold_ + 1
    feature_importance_df = pd.concat([feature_importance_df, fold_importance_df], axis=0)
    
    oof_test_skf[fold_,:]= clf.predict(df_test[features], num_iteration=clf.best_iteration)    
oof_test = oof_test_skf.mean(axis=0)
mean_loss=np.sqrt(mean_squared_error(oof_train.reshape(-1), target))


fold 0
Training until validation scores don't improve for 500 rounds.
[200]	training's rmse: 3.57062	valid_1's rmse: 3.679
[400]	training's rmse: 3.47627	valid_1's rmse: 3.65675
[600]	training's rmse: 3.41366	valid_1's rmse: 3.65078
[800]	training's rmse: 3.36752	valid_1's rmse: 3.6494
[1000]	training's rmse: 3.32629	valid_1's rmse: 3.64918
[1200]	training's rmse: 3.28872	valid_1's rmse: 3.6483
[1400]	training's rmse: 3.25355	valid_1's rmse: 3.64728
[1600]	training's rmse: 3.21951	valid_1's rmse: 3.64686
[1800]	training's rmse: 3.18771	valid_1's rmse: 3.64714
[2000]	training's rmse: 3.1572	valid_1's rmse: 3.64761
[2200]	training's rmse: 3.12821	valid_1's rmse: 3.648
Early stopping, best iteration is:
[1711]	training's rmse: 3.20118	valid_1's rmse: 3.64674
fold 1
Training until validation scores don't improve for 500 rounds.
[200]	training's rmse: 3.57168	valid_1's rmse: 3.67266
[400]	training's rmse: 3.47629	valid_1's rmse: 3.65279
[600]	training's rmse: 3.41396	valid_1's rmse: 3.64791

In [22]:
mean_loss=np.sqrt(mean_squared_error(oof_train.reshape(-1), target))
print(mean_loss)
train_prob=pd.DataFrame(oof_train)
train_prob.columns=['class1']
train_prob.to_csv("oof/train_prob_%s.csv"%mean_loss,index=False)

test_prob=pd.DataFrame(oof_test)
test_prob.columns=['class1']
test_prob.to_csv("oof/test_prob_%s.csv"%mean_loss,index=False)

np.save("train_y",target)

3.6477934509677534


In [23]:
#cv: 3.6497796168629439  lb:3.693
cols = (feature_importance_df[["Feature", "importance"]]
        .groupby("Feature")
        .mean()
        .sort_values(by="importance", ascending=False)[:500].index)

best_features = feature_importance_df.loc[feature_importance_df.Feature.isin(cols)]

plt.figure(figsize=(14,25))
sns.barplot(x="importance",
            y="Feature",
            data=best_features.sort_values(by="importance",
                                           ascending=False))
plt.title('LightGBM Features (avg over folds)')
plt.tight_layout()
plt.savefig('lgbm_importances.png')

In [24]:
sub_df = pd.DataFrame({"card_id":df_test["card_id"].values})
sub_df["target"] = oof_test
sub_df.to_csv("sub/submission_%s.csv"%mean_loss, index=False)

In [25]:
feature_importance_df.sort_values(by='importance',inplace=True)
feature_importance_df

,Feature,importance,fold
85,new_hist_year_mean_mean,2,5
53,new_hist_month_nunique,2,5
53,new_hist_month_nunique,3,2
53,new_hist_month_nunique,4,4
53,new_hist_month_nunique,4,3
85,new_hist_year_mean_mean,4,2
85,new_hist_year_mean_mean,5,4
105,new_hist_year_nunique_add_hist_year_nunique,5,4
85,new_hist_year_mean_mean,6,1
105,new_hist_year_nunique_add_hist_year_nunique,6,5


In [26]:
# #feature select

# def get_feature_importances(data, shuffle, seed=None):
#     # Gather real features
#     train_features = [f for f in data if f not in ['card_id', 'first_active_month','target','outliers']]
#     # Go over fold and keep track of CV score (train and valid) and feature importances
    
#     # Shuffle target if required
#     y = data['target'].copy()
#     if shuffle:
#         # Here you could as well use a binomial distribution
#         y = data['target'].copy().sample(frac=1.0)
    
#     # Fit LightGBM in RF mode, yes it's quicker than sklearn RandomForest
#     dtrain = lgb.Dataset(data[train_features], y, free_raw_data=False, silent=True)
#     lgb_params = {
#         'objective': 'regression',
#         'boosting_type': 'rf',
#         'subsample': 0.623,
#         'colsample_bytree': 0.7,
#         'num_leaves': 127,
#         'max_depth': 8,
#         'seed': 4590,
#         'bagging_freq': 1,
#         'n_jobs': 4
#     }
    
#     # Fit the model
#     clf = lgb.train(params=lgb_params, train_set=dtrain, num_boost_round=200)

#     # Get feature importances
#     imp_df = pd.DataFrame()
#     imp_df["feature"] = list(train_features)
#     imp_df["importance_gain"] = clf.feature_importance(importance_type='gain')
#     imp_df["importance_split"] = clf.feature_importance(importance_type='split')
    
#     return imp_df


# # Seed the unexpected randomness of this world
# np.random.seed(123)
# # Get the actual importance, i.e. without shuffling
# actual_imp_df = get_feature_importances(data=df_train, shuffle=False)


# null_imp_df = pd.DataFrame()
# nb_runs = 80
# import time
# start = time.time()
# dsp = ''
# for i in range(nb_runs):
#     # Get current run importances
#     imp_df = get_feature_importances(data=df_train, shuffle=True)
#     imp_df['run'] = i + 1 
#     # Concat the latest importances with the old ones
#     null_imp_df = pd.concat([null_imp_df, imp_df], axis=0)
#     # Erase previous message
#     for l in range(len(dsp)):
#         print('\b', end='', flush=True)
#     # Display current run and time used
#     spent = (time.time() - start) / 60
#     dsp = 'Done with %4d of %4d (Spent %5.1f min)' % (i + 1, nb_runs, spent)
#     print(dsp, end='', flush=True)

# """
# Score features
# There are several ways to score features :

# Compute the number of samples in the actual importances that are away from the null importances recorded distribution.
# Compute ratios like Actual / Null Max, Actual / Null Mean, Actual Mean / Null Max
# In a first step I will use the log actual feature importance divided by the 75 percentile of null distribution.

# """
    
# feature_scores = []
# for _f in actual_imp_df['feature'].unique():
#     f_null_imps_gain = null_imp_df.loc[null_imp_df['feature'] == _f, 'importance_gain'].values
#     f_act_imps_gain = actual_imp_df.loc[actual_imp_df['feature'] == _f, 'importance_gain'].mean()
#     gain_score = np.log(1e-10 + f_act_imps_gain / (1 + np.percentile(f_null_imps_gain, 75)))  # Avoid didvide by zero
#     f_null_imps_split = null_imp_df.loc[null_imp_df['feature'] == _f, 'importance_split'].values
#     f_act_imps_split = actual_imp_df.loc[actual_imp_df['feature'] == _f, 'importance_split'].mean()
#     split_score = np.log(1e-10 + f_act_imps_split / (1 + np.percentile(f_null_imps_split, 75)))  # Avoid didvide by zero
#     feature_scores.append((_f, split_score, gain_score))

# scores_df = pd.DataFrame(feature_scores, columns=['feature', 'split_score', 'gain_score'])

# thr_list=scores_df['gain_score'].tolist()
# thr_list=[ i for i in set(thr_list)]
# thr_list=sorted(thr_list)


# def test_feature(test_fea):
#     kf = StratifiedKFold(n_splits=5, shuffle=True, random_state=4590)
#     train_x = df_train[test_fea].values
#     train_y = df_train['target']
#     oof = np.zeros((train_x.shape[0],1))
#     df_train.reset_index(drop=True,inplace=True)
#     for idx,(idx_trn,idx_val) in enumerate(kf.split(train_x,df_train['outliers'].values)):
#         print('第 %d fold'%idx)
#         tr_x,tr_y,val_x,val_y=train_x[idx_trn],train_y[idx_trn],train_x[idx_val],train_y[idx_val]

#         trn_data = lgb.Dataset(tr_x,tr_y)#, categorical_feature=categorical_feats)
#         val_data = lgb.Dataset(val_x,val_y)#, categorical_feature=categorical_feats)

#         clf = lgb.train(param, trn_data, num_round, valid_sets = [trn_data, val_data], verbose_eval=200, early_stopping_rounds = 500)
#         oof[idx_val] = clf.predict(val_x, num_iteration=clf.best_iteration).reshape(-1,1)

#     loss=np.sqrt(mean_squared_error(oof.reshape(-1), target))
#     print('mean loss %f'%loss)
#     return loss


# test_dict={}
# for idx,i in enumerate(thr_list):
#     if idx>50:
#         break
#     print(i)
#     bed_fea=set(scores_df[scores_df['gain_score']<=i]['feature'].tolist())
#     bed_fea=[i for i in bed_fea]
#     print(bed_fea)
#     del_col =  ['card_id', 'first_active_month','target','outliers']+bed_fea
#     df_test_fea = [c for c in df_train.columns if c not in del_col ]
#     test_dict[''.join(bed_fea)]=test_feature(df_test_fea)

In [27]:
# min(test_dict.values())

In [28]:
df_hist_ca=['authorized_flag','city_id','category_1','installments','category_3','merchant_category_id','merchant_id','category_2','state_id',\
          'subsector_id','month_lag','purchase_amount' ]

hist_text_feature={}
gp=df_hist_trans.groupby('card_id')
for fea  in df_hist_ca:
    hist_text_feature[fea+"_hist_text"]=[]
hist_text_feature['card_id']=[]
for cid,cid_df in  gp:
    hist_text_feature['card_id'].append(cid)
    for  fea in  df_hist_ca:
        if fea=='card_id':
            continue
        hist_text_feature[fea+"_hist_text"].append(" ".join(cid_df[fea].astype(str).tolist()))
df_hist_text=pd.DataFrame(hist_text_feature)


new_mer_text_feature={}
gp=df_new_merchant_trans.groupby('card_id')
for fea  in df_hist_ca:
    new_mer_text_feature[fea+"_new_mer_text"]=[]
new_mer_text_feature['card_id']=[]
for cid,cid_df in  gp:
    new_mer_text_feature['card_id'].append(cid)
    for  fea in  df_hist_ca:
        if fea=='card_id':
            continue
        new_mer_text_feature[fea+"_new_mer_text"].append(" ".join(cid_df[fea].astype(str).tolist()))
df_new_mer_text=pd.DataFrame(new_mer_text_feature)

In [ ]:
df_train=pd.merge(df_train,df_hist_text,how='left',on='card_id')
df_test=pd.merge(df_test,df_hist_text,how='left',on='card_id')

df_all=pd.concat([df_train,df_test])

text_fea=[i for i in df_all.columns if "text"in i]

from sklearn.feature_extraction.text import CountVectorizer
from scipy import sparse


cv=CountVectorizer(analyzer='word',token_pattern=u"(?u)\\b\\w+\\b")



for idx,i in enumerate(text_fea):
    cv.fit(df_all[i])
    tr_x=cv.transform(df_train[i])
    te_x=cv.transform(df_test[i])
    
    if idx==0:
        cv_Train_x=tr_x
        cv_Test_x=te_x
    else:
        cv_Train_x=sparse.hstack((cv_Train_x,tr_x))
        cv_Test_x=sparse.hstack((cv_Test_x,te_x))
    
print(cv_Train_x.shape,cv_Test_x.shape)


(201917, 494482) (123623, 494482)


In [ ]:

oof = np.zeros(len(df_train))
predictions = np.zeros(len(df_test))
feature_importance_df = pd.DataFrame()

train_x=df_train[features].values
test_x=df_test[features].values
train_y=df_train['target'].values

train_x=sparse.hstack((train_x,cv_Train_x),'csr')
test_x=sparse.hstack((test_x,cv_Test_x),'csr')
oof_train2 = np.zeros((train_x.shape[0],1))
oof_test2 = np.zeros((test_x.shape[0],1))
oof_test2_skf = np.zeros((NFOLD,test_x.shape[0],))

print(train_x.shape,test_x.shape)

feature_importance_df = pd.DataFrame()

for fold_, (trn_idx, val_idx) in enumerate(skf.split(train_x,df_train['outliers'].values)):
    print("fold {}".format(fold_))
    x_tr,y_tr,x_te,y_te=train_x[trn_idx],train_y[trn_idx],train_x[val_idx],train_y[val_idx]
    trn_data = lgb.Dataset(x_tr,y_tr)
    val_data = lgb.Dataset(x_te,y_te)
    num_round = 10000
    clf = lgb.train(param, trn_data, num_round, valid_sets = [trn_data, val_data], verbose_eval=200, early_stopping_rounds = 500)
    oof_train2[val_idx] = clf.predict(x_te, num_iteration=clf.best_iteration).reshape(-1,1)

    oof_test2_skf[fold_,:]= clf.predict(test_x, num_iteration=clf.best_iteration)    
oof_test2 = oof_test2_skf.mean(axis=0)
mean_loss=np.sqrt(mean_squared_error(oof_train2.reshape(-1), target))
print(mean_loss)

(201917, 494735) (123623, 494735)
fold 0
Training until validation scores don't improve for 500 rounds.
[200]	training's rmse: 3.56189	valid_1's rmse: 3.6767
[400]	training's rmse: 3.46522	valid_1's rmse: 3.65441
[600]	training's rmse: 3.40429	valid_1's rmse: 3.64805
[800]	training's rmse: 3.36059	valid_1's rmse: 3.64647
[1000]	training's rmse: 3.32259	valid_1's rmse: 3.64546
[1200]	training's rmse: 3.28723	valid_1's rmse: 3.64566
[1400]	training's rmse: 3.2558	valid_1's rmse: 3.64566
[1600]	training's rmse: 3.22472	valid_1's rmse: 3.64585
Early stopping, best iteration is:
[1116]	training's rmse: 3.30158	valid_1's rmse: 3.64527
fold 1
Training until validation scores don't improve for 500 rounds.
[200]	training's rmse: 3.56206	valid_1's rmse: 3.67388
[400]	training's rmse: 3.46564	valid_1's rmse: 3.65588
[600]	training's rmse: 3.40516	valid_1's rmse: 3.65053
[800]	training's rmse: 3.36012	valid_1's rmse: 3.64889
[1000]	training's rmse: 3.32156	valid_1's rmse: 3.64771
[1200]	training's

In [ ]:
sub_df = pd.DataFrame({"card_id":df_test["card_id"].values})
sub_df["target"] = oof_test2
sub_df.to_csv("sub/submission_%s.csv"%mean_loss, index=False)
mean_loss=np.sqrt(mean_squared_error(oof_train2.reshape(-1), target))
print(mean_loss)
train_prob=pd.DataFrame(oof_train2)
train_prob.columns=['class1']
train_prob.to_csv("oof/train_prob_%s.csv"%mean_loss,index=False)

test_prob=pd.DataFrame(oof_test2)
test_prob.columns=['class1']
test_prob.to_csv("oof/test_prob_%s.csv"%mean_loss,index=False)

np.save("train_y",target)

3.6486189797285364


In [ ]:
df_train=pd.merge(df_train,df_new_mer_text,how='left',on='card_id')
df_test=pd.merge(df_test,df_new_mer_text,how='left',on='card_id')

df_all=pd.concat([df_train,df_test])

text_fea=[i for i in df_all.columns if "new_mer_text"in i]

from sklearn.feature_extraction.text import CountVectorizer
from scipy import sparse


cv=CountVectorizer(analyzer='word',token_pattern=u"(?u)\\b\\w+\\b")



for idx,i in enumerate(text_fea):
    df_all[i].fillna('-1',inplace=True)
    df_train[i].fillna('-1',inplace=True)
    df_test[i].fillna('-1',inplace=True)
    cv.fit(df_all[i])
    tr_x=cv.transform(df_train[i])
    te_x=cv.transform(df_test[i])
    
    if idx==0:
        cv_Train_x_2=tr_x
        cv_Test_x_2=te_x
    else:
        cv_Train_x_2=sparse.hstack((cv_Train_x_2,tr_x))
        cv_Test_x_2=sparse.hstack((cv_Test_x_2,te_x))
    
print(cv_Train_x_2.shape,cv_Test_x_2.shape)



oof = np.zeros(len(df_train))
predictions = np.zeros(len(df_test))
feature_importance_df = pd.DataFrame()

train_x=df_train[features].values
test_x=df_test[features].values
train_y=df_train['target'].values

train_x=sparse.hstack((train_x,cv_Train_x_2),'csr')
test_x=sparse.hstack((test_x,cv_Test_x_2),'csr')
oof_train3 = np.zeros((train_x.shape[0],1))
oof_test3 = np.zeros((test_x.shape[0],1))
oof_test3_skf = np.zeros((NFOLD,test_x.shape[0],))

print(train_x.shape,test_x.shape)

feature_importance_df = pd.DataFrame()

for fold_, (trn_idx, val_idx) in enumerate(skf.split(train_x,df_train['outliers'].values)):
    print("fold {}".format(fold_))
    x_tr,y_tr,x_te,y_te=train_x[trn_idx],train_y[trn_idx],train_x[val_idx],train_y[val_idx]
    trn_data = lgb.Dataset(x_tr,y_tr)
    val_data = lgb.Dataset(x_te,y_te)
    num_round = 10000
    clf = lgb.train(param, trn_data, num_round, valid_sets = [trn_data, val_data], verbose_eval=200, early_stopping_rounds = 500)
    oof_train3[val_idx] = clf.predict(x_te, num_iteration=clf.best_iteration).reshape(-1,1)

    oof_test3_skf[fold_,:]= clf.predict(test_x, num_iteration=clf.best_iteration)    
oof_test3 = oof_test3_skf.mean(axis=0)
mean_loss=np.sqrt(mean_squared_error(oof_train3.reshape(-1), target))
print(mean_loss)


sub_df = pd.DataFrame({"card_id":df_test["card_id"].values})
sub_df["target"] = oof_test3
sub_df.to_csv("sub/submission_%s.csv"%mean_loss, index=False)
mean_loss=np.sqrt(mean_squared_error(oof_train3.reshape(-1), target))
print(mean_loss)
train_prob=pd.DataFrame(oof_train3)
train_prob.columns=['class1']
train_prob.to_csv("oof/train_prob_%s.csv"%mean_loss,index=False)

test_prob=pd.DataFrame(oof_test3)
test_prob.columns=['class1']
test_prob.to_csv("oof/test_prob_%s.csv"%mean_loss,index=False)

np.save("train_y",target)

(201917, 233541) (123623, 233541)
(201917, 233794) (123623, 233794)
fold 0
Training until validation scores don't improve for 500 rounds.
[200]	training's rmse: 3.56898	valid_1's rmse: 3.67825
[400]	training's rmse: 3.4741	valid_1's rmse: 3.6575
[600]	training's rmse: 3.41348	valid_1's rmse: 3.65248
[800]	training's rmse: 3.36601	valid_1's rmse: 3.65028
[1000]	training's rmse: 3.32488	valid_1's rmse: 3.64848
[1200]	training's rmse: 3.2867	valid_1's rmse: 3.64837
[1400]	training's rmse: 3.25186	valid_1's rmse: 3.64804
[1600]	training's rmse: 3.21877	valid_1's rmse: 3.64809
[1800]	training's rmse: 3.18683	valid_1's rmse: 3.64799
[2000]	training's rmse: 3.1579	valid_1's rmse: 3.6487
[2200]	training's rmse: 3.12957	valid_1's rmse: 3.64942
Early stopping, best iteration is:
[1830]	training's rmse: 3.18238	valid_1's rmse: 3.64791
fold 1
Training until validation scores don't improve for 500 rounds.
[200]	training's rmse: 3.57008	valid_1's rmse: 3.67546
[400]	training's rmse: 3.47512	valid_1'

In [ ]:


from sklearn.feature_extraction.text import CountVectorizer
from scipy import sparse



oof = np.zeros(len(df_train))
predictions = np.zeros(len(df_test))
feature_importance_df = pd.DataFrame()

train_x=df_train[features].values
test_x=df_test[features].values
print(train_x.shape,test_x.shape)
train_y= df_train['target'].values
train_x = sparse.hstack((train_x,cv_Train_x),'csr')
test_x = sparse.hstack((test_x,cv_Test_x),'csr')
print(train_x.shape,test_x.shape)

train_x=sparse.hstack((train_x,cv_Train_x_2),'csr')
test_x=sparse.hstack((test_x,cv_Test_x_2),'csr')





oof_train4 = np.zeros((train_x.shape[0],1))
oof_test4 = np.zeros((test_x.shape[0],1))
oof_test4_skf = np.zeros((NFOLD,test_x.shape[0],))

print(train_x.shape,test_x.shape)

feature_importance_df = pd.DataFrame()

for fold_, (trn_idx, val_idx) in enumerate(skf.split(train_x,df_train['outliers'].values)):
    print("fold {}".format(fold_))
    

    x_tr,y_tr,x_te,y_te=train_x[trn_idx],train_y[trn_idx],train_x[val_idx],train_y[val_idx]
    trn_data = lgb.Dataset(x_tr,y_tr)
    val_data = lgb.Dataset(x_te,y_te)
    num_round = 10000
    clf = lgb.train(param, trn_data, num_round, valid_sets = [trn_data, val_data], verbose_eval=200, early_stopping_rounds = 500)
    oof_train4[val_idx] = clf.predict(x_te, num_iteration=clf.best_iteration).reshape(-1,1)

    oof_test4_skf[fold_,:]= clf.predict(test_x, num_iteration=clf.best_iteration)    
oof_test4 = oof_test4_skf.mean(axis=0)
mean_loss=np.sqrt(mean_squared_error(oof_train4.reshape(-1), target))
print(mean_loss)


sub_df = pd.DataFrame({"card_id":df_test["card_id"].values})
sub_df["target"] = oof_test4
sub_df.to_csv("sub/submission_%s.csv"%mean_loss, index=False)
mean_loss=np.sqrt(mean_squared_error(oof_train4.reshape(-1), target))
print(mean_loss)
train_prob=pd.DataFrame(oof_train4)
train_prob.columns=['class1']
train_prob.to_csv("oof/train_prob_%s.csv"%mean_loss,index=False)

test_prob=pd.DataFrame(oof_test4)
test_prob.columns=['class1']
test_prob.to_csv("oof/test_prob_%s.csv"%mean_loss,index=False)

np.save("train_y",target)

(201917, 253) (123623, 253)
(201917, 494735) (123623, 494735)
(201917, 728276) (123623, 728276)
fold 0
Training until validation scores don't improve for 500 rounds.
[200]	training's rmse: 3.56265	valid_1's rmse: 3.67707
[400]	training's rmse: 3.46449	valid_1's rmse: 3.65396
[600]	training's rmse: 3.40546	valid_1's rmse: 3.64799
[800]	training's rmse: 3.36073	valid_1's rmse: 3.64678
[1000]	training's rmse: 3.3234	valid_1's rmse: 3.64735
[1200]	training's rmse: 3.28849	valid_1's rmse: 3.64676
[1400]	training's rmse: 3.25596	valid_1's rmse: 3.647
[1600]	training's rmse: 3.22572	valid_1's rmse: 3.64757
Early stopping, best iteration is:
[1254]	training's rmse: 3.27977	valid_1's rmse: 3.64637
fold 1
Training until validation scores don't improve for 500 rounds.
[200]	training's rmse: 3.56243	valid_1's rmse: 3.67414
[400]	training's rmse: 3.46314	valid_1's rmse: 3.65449
[600]	training's rmse: 3.40402	valid_1's rmse: 3.6491
[800]	training's rmse: 3.3593	valid_1's rmse: 3.64731
[1000]	trainin

In [ ]:
trn_idx

array([     0,      1,      2, ..., 201912, 201913, 201916])

In [ ]:
from sklearn.linear_model import BayesianRidge

train_stack = np.concatenate([oof_train,oof_train2,oof_train3,oof_train4],axis=1)
test_stack = np.concatenate([oof_test.reshape(-1,1),oof_test2.reshape(-1,1),\
                             oof_test3.reshape(-1,1),oof_test4.reshape(-1,1)],axis=1)
print(train_stack.shape,test_stack.shape)

oof_stack = np.zeros(train_stack.shape[0])
predictions_3 = np.zeros(test_stack.shape[0])

for fold_, (trn_idx, val_idx) in enumerate(skf.split(train_stack,df_train['outliers'].values)):
    print("fold {}".format(fold_))

    trn_data, trn_y = train_stack[trn_idx], target.iloc[trn_idx].values
    val_data, val_y = train_stack[val_idx], target.iloc[val_idx].values
    
    clf_3 = BayesianRidge()
    clf_3.fit(trn_data, trn_y)
    
    oof_stack[val_idx] = clf_3.predict(val_data)
    predictions_3 += clf_3.predict(test_stack) / 5
    
stack_loss=np.sqrt(mean_squared_error(target.values, oof_stack))
print(stack_loss)
sub_df = pd.DataFrame({"card_id":df_test["card_id"].values})
sub_df["target"] = predictions_3
sub_df.to_csv("sub/submission_%s.csv"%stack_loss, index=False)

train_prob=pd.DataFrame(oof_stack)
train_prob.columns=['class1']
train_prob.to_csv("oof/train_prob_%s.csv"%stack_loss,index=False)

test_prob=pd.DataFrame(predictions_3)
test_prob.columns=['class1']
test_prob.to_csv("oof/test_prob_%s.csv"%stack_loss,index=False)

(201917, 4) (123623, 4)
fold 0
fold 1
fold 2
fold 3
fold 4
3.6419621998951706
